In [8]:
import numpy as np
import random
import tensorflow as tf
from numpy import linalg as LA

def tau(s,a):
    if s==0 or s==4:  return(s)
    else:      return(s+a)
def rho(s,a):
    return(s==1 and a==0)+2*(s==3 and a==1)
def calc_policy(Q):
    policy=np.zeros(5)
    for s in range(0,5):
        action_idx=np.argmax(Q[s,:])
        policy[s]=2*action_idx-1
        policy[0]=policy[4]=0
    return policy.astype(int)
def idx(a):
    return(int((a+1)/2))

inputs = tf.placeholder(shape=[1,5],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([5,2],0,0.01))
Qout = tf.matmul(inputs,W)
predict = tf.argmax(Qout,1)    
nextQ = tf.placeholder(shape=[1,2],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

init = tf.global_variables_initializer()

y = 0.03
e = 0.1
epi_num = 1000
with tf.Session() as sess:
    sess.run(init)
    for i in range(epi_num):
        s = np.random.randint(0,5)
        j = 0
        while j < 5: 
            a,Qall = sess.run([predict,Qout],feed_dict={inputs:np.identity(5)[s:s+1]})
            s_new = tau(s,a[0])
            Q1 = sess.run(Qout,feed_dict={inputs:np.identity(5)[s_new:s_new+1]})
            maxQ1 = np.max(Q1)
            targetQ = Qall 
            r = rho(s,a[0])
            targetQ[0,a[0]] = r + y*maxQ1
            sess.run([updateModel,W],feed_dict={inputs:np.identity(5)[s:s+1],nextQ:targetQ})
            s = s_new
            j+=1
    result = sess.run(W)
    policy = calc_policy(tf.global_variables()[0].eval())
    print('Policy'+str(policy))
    print('Q values: \n',np.transpose(np.around(result, decimals=1)))


Policy[ 0 -1  1  1  0]
Q values: 
 [[0.  1.  0.  0.  0. ]
 [0.  0.  0.1 2.  0. ]]
